In [0]:
from pyspark.sql.functions import to_date, first, col, round

In [0]:
df_junto = spark.read.parquet("dbfs:/databricks-results/bronze/*/*/*")

+-----+----------+----------+
|moeda|      taxa|      data|
+-----+----------+----------+
|  SBD|  1.523375|2025-06-28|
|  SCR|  2.676377|2025-06-28|
|  SDG|109.590878|2025-06-28|
|  SEK|   1.73045|2025-06-28|
|  SGD|   0.23285|2025-06-28|
+-----+----------+----------+
only showing top 5 rows


In [0]:
moedas = ['USD', 'EUR', 'GBP']

df_moedas = df_junto.filter(df_junto.moeda.isin(moedas))

+-----+--------+----------+
|moeda|    taxa|      data|
+-----+--------+----------+
|  USD|0.182498|2025-06-28|
|  USD|0.184104|2025-06-30|
|  USD| 0.18314|2025-07-01|
|  USD|0.182485|2025-06-29|
|  USD| 0.17963|2025-07-21|
+-----+--------+----------+
only showing top 5 rows


In [0]:


df_moedas = df_moedas.withColumn("data", to_date("data"))

In [0]:


resultado_taxas_conversao = df_moedas.groupby("data")\
                                    .pivot("moeda")\
                                    .agg(first("taxa"))\
                                    .orderBy("data",ascending=False)

+----------+--------+--------+--------+
|      data|     EUR|     GBP|     USD|
+----------+--------+--------+--------+
|2025-06-28|0.155727|  0.1347|0.182498|
|2025-06-29|0.155612|0.133018|0.182485|
|2025-06-30|0.156198|0.134092|0.184104|
|2025-07-01|0.155134|0.133236| 0.18314|
|2025-07-18|0.154178|0.133598|0.179195|
+----------+--------+--------+--------+
only showing top 5 rows


In [0]:
resultado_valores_reais = resultado_taxas_conversao.select("*") 

+----------+--------+--------+--------+
|      data|     EUR|     GBP|     USD|
+----------+--------+--------+--------+
|2025-06-28|0.155727|  0.1347|0.182498|
|2025-06-29|0.155612|0.133018|0.182485|
|2025-06-30|0.156198|0.134092|0.184104|
|2025-07-01|0.155134|0.133236| 0.18314|
|2025-07-18|0.154178|0.133598|0.179195|
+----------+--------+--------+--------+
only showing top 5 rows


In [0]:


for moeda in moedas:
    resultado_valores_reais = resultado_valores_reais\
                                .withColumn(
                                    moeda, round(1/col(moeda),4)
                                    )

In [0]:
resultado_taxas_conversao = resultado_taxas_conversao.coalesce(1)
resultado_valores_reais = resultado_valores_reais.coalesce(1)

In [0]:
resultado_taxas_conversao.write\
    .mode ("overwrite")\
    .format("csv")\
    .option("header", "true")\
    .save("dbfs:/databricks-results/prata/taxas_conversao")
    
resultado_valores_reais.write\
    .mode ("overwrite")\
    .format("csv")\
    .option("header", "true")\
    .save("dbfs:/databricks-results/prata/valores_reais")

In [0]:
dbutils.fs.ls("dbfs:/databricks-results/bronze/2025/09/04")

[FileInfo(path='dbfs:/databricks-results/bronze/2025/09/04/_committed_2025951854436089086', name='_committed_2025951854436089086', size=823, modificationTime=1757034438000),
 FileInfo(path='dbfs:/databricks-results/bronze/2025/09/04/_committed_2447108880190010500', name='_committed_2447108880190010500', size=823, modificationTime=1757037253000),
 FileInfo(path='dbfs:/databricks-results/bronze/2025/09/04/_committed_6213096474060937244', name='_committed_6213096474060937244', size=834, modificationTime=1757034399000),
 FileInfo(path='dbfs:/databricks-results/bronze/2025/09/04/_committed_6562030111697256892', name='_committed_6562030111697256892', size=424, modificationTime=1757034362000),
 FileInfo(path='dbfs:/databricks-results/bronze/2025/09/04/_committed_vacuum9021653038760063555', name='_committed_vacuum9021653038760063555', size=96, modificationTime=1757037254000),
 FileInfo(path='dbfs:/databricks-results/bronze/2025/09/04/_started_2447108880190010500', name='_started_24471088801900